In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [2]:
# 展平每个图片，变为长度784的向量，每个分量都是灰度值，因为我们数据集有10个类别，所以输出维度是10

num_inputs = 784
num_outputs = 10

W = torch.normal(0,0.01,(num_inputs,num_outputs),requires_grad=True)
b = torch.zeros(num_outputs,requires_grad=True)

In [3]:
# 实现softmax操作
# 这里的X不再是个行向量，是个矩阵，每行代表一个样本的输出结果，10列
def softmax(X):
    X_exp = torch.exp(X) # 张量的每个分量做了e^x
    partition = X_exp.sum(dim=1,keepdim=True) # 按列相加，得到每个样本的总e^x和
    return X_exp/partition  # 广播机制

In [ ]:
# 实现softmax模型
# X为 256个图片 * 784灰度值的矩阵，结果是256*10的矩阵
def net(X):
    return softmax(torch.matmul(X,W)+b)   # +b是利用了广播机制

In [4]:
# 一个列表可以作为另一个列表的索引
y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat[[0,1],[0,2]] # 表示取 0,0  和 1,2 坐标的数据，就是（x,y)坐标数据，第一个入参是x的数据

tensor([0.1000, 0.5000])

In [8]:
# 实现交叉熵损失函数 -log(y_hat对应y中正确的)  y向量的长度对应y_hat的行数，y的每一个分量值对应某一行的列坐标
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat),y)]) 
# 根据上面cell的原理，256行每一行都存在对应一个真实结果，所以x有0,1...255
# 对应的y则去匹配x行的某一列，且y是256*1的向量，如果y是 3,1,0,4,0,9.. 那么结果是(0,3),(1,1),(2,0),(3,4).....的y_hat结果向量
# 返回256*1的损失结果

In [9]:
# 预测值和真实值比较，得到正确判断的数量
def accuracy(y_hat,y):
    if len(y_hat.shape)>1 and len(y_hat.shape[1])>1: # 如果大于一行且大于1列
        y_hat = y_hat.argmax(axis=1) # 每一行都得到最大值的那个列下标，比如第一行最大值在第三列，结果就是y_hat[0]=3，意思是第一个样本预测值为3
    cmp = y_hat.type(y.dtype)==y  #y_hat转成y的数据类型，并且比较两个向量，得到一个都是 true,false组成的张量
    return float(cmp.type(y.dtype).sum())  # true代表1，sum()会把所有true求和